In [1]:
import re
import pdfplumber
import pandas as pd
from pprint import pprint as pp
from geopy.distance import distance
from geopy.geocoders import Nominatim

In [ ]:
pdf_path = r'C:\Users\Roshi\Documents\Git_Repos\la_tax_auction_parse\data\2025B-Auction-Book.pdf'
csv_path = r'C:\Users\Roshi\Downloads\Assessor_Parcels_Data_-_2006_thru_2021_output.csv'
tables = []
df = pd.DataFrame()
# Setup geopy
geolocator = Nominatim(user_agent="assessor_parse")

In [3]:
with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        table_data = page.extract_table({
            "vertical_strategy": "lines", 
            "horizontal_strategy": "lines",
        })

        if table_data:
            clean_data = [x for x in table_data if x.count('') != len(x)]
            if len(clean_data) > 1:
                df_page = pd.DataFrame(clean_data[1:], columns=clean_data[0])
                tables.append(df_page)

if tables:
    df = pd.concat(tables, ignore_index=True)

In [4]:
def clean_address(addr):
    # Remove newlines
    addr = addr.replace("\n", " ")
    # Collapse multiple spaces
    addr = re.sub(r"\s+", " ", addr)
    # Remove trailing 4-digit extension after ZIP (e.g. "91303 1334" → "91303")
    addr = re.sub(r"(\d{5})\s\d{4}$", r"\1", addr)
    # Trim spaces
    return addr.strip()

def get_lat_long(addr):
    if pd.isna(addr) or addr.strip().upper() == "VACANT LOT":
        return None, None   # skip these
    try:
        location = geolocator.geocode(addr)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {addr}: {e}")
    return None, None

In [12]:
# Apply function to new column
df["clean_address"] = df["PROPERTY ADDRESS"].apply(clean_address)

# Apply function → returns tuple
df[["Latitude", "Longitude"]] = df["clean_address"].apply(
    lambda x: pd.Series(get_lat_long(x))
)

Error geocoding 7348 JORDAN AVE LOS ANGELES CA 91303: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=7348+JORDAN+AVE+LOS+ANGELES+CA+91303&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding 7308 CORBIN AVE UNIT F LOS ANGELES CA 91335: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=7308+CORBIN+AVE+UNIT+F+LOS+ANGELES+CA+91335&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding 18415 COLLINS ST UNIT C LOS ANGELES CA 91356: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=18415+COLLINS+ST+UNIT+C+LOS+ANGELES+CA+91356&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(

In [ ]:
latdf[df['clean_address'].str.contains('STAGG ', na=False)].head(50)
# print(len(df))

,ITEM,AIN,MIN BID,IMP,NSB#,LEGAL DESCRIPTION,LOCATION,PROPERTY ADDRESS,clean_address
5,1479,2020-027-022,"$30,438",Y,536,*TR=37199 LOT 11,CITY-LOS ANGELES,23901 STAGG ST LOS ANGELES CA 91304 6115,23901 STAGG ST LOS ANGELES CA 91304


In [16]:
df_dict = df.to_dict('records')

In [ ]:
current = (33.76403641974573, -118.2012596022652)

for loc in df_dict:
    if loc['Latitude'] and loc['Longitude']:
        print(loc['Latitude'], loc['Longitude'])
        loc_coord = (float(loc['Latitude']), float(loc['Longitude']))
        loc['distance'] = distance(current, loc_coord).miles
    


nan nan


ValueError: Point coordinates must be finite. (nan, nan, 0.0) has been passed as coordinates.

In [15]:
df.to_csv(r'C:\Users\Roshi\Documents\Git_Repos\la_tax_auction_parse\data\2025B-Auction-Book.csv', index=False)

In [13]:
df.head(50)

,ITEM,AIN,MIN BID,IMP,NSB#,LEGAL DESCRIPTION,LOCATION,PROPERTY ADDRESS,clean_address,Latitude,Longitude
0,1472,2017-002-036,"$9,459",,536,TRACT # 9683 LOT 22,CITY-LOS ANGELES,VACANT LOT,VACANT LOT,NaN,NaN
1,1473,2017-002-040,"$7,959",,536,TRACT # 9683 LOT 26,CITY-LOS ANGELES,VACANT LOT,VACANT LOT,NaN,NaN
2,1474,2017-014-029,"$18,384",,536,TRACT # 4034 LOT COM AT MOST S COR OF LOT 62 T...,CITY-LOS ANGELES,8315 HILLCROFT DR LOS ANGELES CA 91304 2113,8315 HILLCROFT DR LOS ANGELES CA 91304,34.223757,-118.650977
3,1475,2017-015-035,"$13,892",,536,TRACT # 9889 LOT 4 BLK E,CITY-LOS ANGELES,8727 CHATLAKE DR LOS ANGELES CA 91304 2106,8727 CHATLAKE DR LOS ANGELES CA 91304,34.228611,-118.646851
4,1477,2017-025-035,"$19,881",,536,TRACT # 7821 E 83.63 FT OF S 100 FT OF LOT 7,CITY-LOS ANGELES,VACANT LOT,VACANT LOT,NaN,NaN
5,1479,2020-027-022,"$30,438",Y,536,*TR=37199 LOT 11,CITY-LOS ANGELES,23901 STAGG ST LOS ANGELES CA 91304 6115,23901 STAGG ST LOS ANGELES CA 91304,34.210783,-118.649386
6,1480,2024-002-096,"$34,419",Y,536,*TR=38920 CONDOMINIUM*UNIT 50,CITY LOS ANGELES,22040 GAULT ST UNIT 50 LOS ANGELES CA 91303\n1871,22040 GAULT ST UNIT 50 LOS ANGELES CA 91303,NaN,NaN
7,1484,2063-019-031,"$17,603",,537,"*LAND DESC IN DOC 0755294,200710*TR= 2804*POR ...",COUNTY OF LOS ANGELES,VACANT LOT,VACANT LOT,NaN,NaN
8,1487,2080-029-044,"$2,607",,537,TR=34173 FOR DESC SEE ASSESSOR'S MAPS POR OF L...,CITY-CALABASAS T,VACANT LOT,VACANT LOT,NaN,NaN
9,1489,2104-002-017,"$112,829",Y,539,TRACT NO 11736 E 81.O3 FT MEASURED ON S L1NE O...,CITY-LOS ANGELES,19106 CANTARA ST LOS ANGELES CA 91335 1112,19106 CANTARA ST LOS ANGELES CA 91335,34.219173,-118.549452
